# Notes for decentmon data processing

In [33]:
# adding required packages if not present
!pip install numpy pandas seaborn matplotlib sklearn

# importing them
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier
from decmon.extractor import *
from decmon.cleaner import *
from decmon.filter import *

In [34]:
# Constants
FILENAME = "sample_log.txt"

In [35]:
# idempotent transformation of input file to csv
# file = open(FILENAME, "rt")
# raw_data = file.read()
# raw_data = raw_data.replace(' ', ',') #add commas to read it as a csv
# file.close()
# file = open(FILENAME, "wt")
# file.write(raw_data)
# file.close()

In [36]:
df = pd.read_csv(FILENAME, sep='@')
df.insert(0, 'formula_id', range(0, len(df)))
df.columns = df.columns.str.strip()
df

,formula_id,x,cent_trace_len,decent_trace_len,odecent_trace_len,cent_num_mess,decent_num_mess,odecent_num_mess,cent_size_mess,decent_size_mess,odecent_size_mess,cent_nb_progressions,decent_nb_progressions,odecent_nb_progressions,formula,trace
0,0,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""b"")",{a|b| } ; { | | } ; {a| | } ; { | |c} ; { |b|...
1,1,1,2,3,2,3,2,0,1.0,1.66,0.00,4,4,4,"Ev (Var ""b"")",{ | |c} ; {a|b|c} ; {a| |c} ; {a|b|c} ; { |b|...
2,2,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""a"")",{ |b|c} ; {a| | } ; {a| |c} ; { | | } ; {a|b|...
3,3,1,6,7,6,15,2,0,1.0,0.55,0.00,12,24,12,"Glob (Var ""b"")",{a|b| } ; {a|b|c} ; { |b| } ; { |b|c} ; {a|b|...
4,4,1,2,3,2,3,2,0,1.0,1.66,0.00,4,8,4,"Glob (Var ""c"")",{a| |c} ; { | | } ; { | | } ; {a| | } ; {a| |...
5,5,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""b"")",{a|b|c} ; { |b| } ; { |b| } ; {a| |c} ; { | |...
6,6,1,4,5,4,9,2,0,1.0,0.83,0.00,8,12,8,"Ev (Var ""c"")",{a| | } ; { |b| } ; {a|b| } ; {a| |c} ; { | |...
7,7,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""a"")",{a|b|c} ; {a| |c} ; {a| | } ; {a|b|c} ; {a| |...
8,8,1,4,5,4,9,2,0,1.0,0.83,0.00,8,16,8,"Glob (Var ""b"")",{ |b| } ; { |b| } ; {a|b| } ; {a| |c} ; { |b|...
9,9,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""c"")",{a|b| } ; {a| |c} ; {a|b|c} ; {a|b| } ; {a|b|...


In [37]:
strategies = {
    "cent" : "orchestration",
    "decent" : "migration",
    "odecent" : "choreography"
}

In [38]:
strategies_df = []
for key, name in strategies.items():
    others = {x: strategies[x] for x in strategies if x != key}
    other_keys = list(others.keys())
    local = rename(df, fr"^{key}_(.*)", r"\1")
    local = exclude_annotate(local, exclude=other_keys, annotate=name)
    strategies_df.append(local)

(cent, decent, odecent) = strategies_df
cent

,formula_id,x,trace_len,num_mess,size_mess,nb_progressions,formula,trace,strategy
0,0,1,2,3,1.0,3,"Next (Var ""b"")",{a|b| } ; { | | } ; {a| | } ; { | |c} ; { |b|...,orchestration
1,1,1,2,3,1.0,4,"Ev (Var ""b"")",{ | |c} ; {a|b|c} ; {a| |c} ; {a|b|c} ; { |b|...,orchestration
2,2,1,2,3,1.0,3,"Next (Var ""a"")",{ |b|c} ; {a| | } ; {a| |c} ; { | | } ; {a|b|...,orchestration
3,3,1,6,15,1.0,12,"Glob (Var ""b"")",{a|b| } ; {a|b|c} ; { |b| } ; { |b|c} ; {a|b|...,orchestration
4,4,1,2,3,1.0,4,"Glob (Var ""c"")",{a| |c} ; { | | } ; { | | } ; {a| | } ; {a| |...,orchestration
5,5,1,2,3,1.0,3,"Next (Var ""b"")",{a|b|c} ; { |b| } ; { |b| } ; {a| |c} ; { | |...,orchestration
6,6,1,4,9,1.0,8,"Ev (Var ""c"")",{a| | } ; { |b| } ; {a|b| } ; {a| |c} ; { | |...,orchestration
7,7,1,2,3,1.0,3,"Next (Var ""a"")",{a|b|c} ; {a| |c} ; {a| | } ; {a|b|c} ; {a| |...,orchestration
8,8,1,4,9,1.0,8,"Glob (Var ""b"")",{ |b| } ; { |b| } ; {a|b| } ; {a| |c} ; { |b|...,orchestration
9,9,1,2,3,1.0,3,"Next (Var ""c"")",{a|b| } ; {a| |c} ; {a|b|c} ; {a|b| } ; {a|b|...,orchestration


In [39]:
sdf = pd.concat(strategies_df, ignore_index=True)
sdf

,formula_id,x,trace_len,num_mess,size_mess,nb_progressions,formula,trace,strategy
0,0,1,2,3,1.00,3,"Next (Var ""b"")",{a|b| } ; { | | } ; {a| | } ; { | |c} ; { |b|...,orchestration
1,1,1,2,3,1.00,4,"Ev (Var ""b"")",{ | |c} ; {a|b|c} ; {a| |c} ; {a|b|c} ; { |b|...,orchestration
2,2,1,2,3,1.00,3,"Next (Var ""a"")",{ |b|c} ; {a| | } ; {a| |c} ; { | | } ; {a|b|...,orchestration
3,3,1,6,15,1.00,12,"Glob (Var ""b"")",{a|b| } ; {a|b|c} ; { |b| } ; { |b|c} ; {a|b|...,orchestration
4,4,1,2,3,1.00,4,"Glob (Var ""c"")",{a| |c} ; { | | } ; { | | } ; {a| | } ; {a| |...,orchestration
...,...,...,...,...,...,...,...,...,...
76,22,3,14,13,2.51,81,"Next (Ev (And (Var ""b"", Var ""c"")))",{ | | } ; { | | } ; {a| | } ; {a| |c} ; { | |...,choreography
77,23,3,16,15,2.57,191,"Ev (Next (And (Var ""a"", Var ""b"")))",{ | |c} ; {a| | } ; { |b|c} ; { |b|c} ; { | |...,choreography
78,24,3,13,11,2.22,123,"Ev (And (Neg (Var ""a""), Next (Var ""c"")))",{a| |c} ; { | | } ; {a| | } ; { | | } ; {a|b|...,choreography
79,25,3,9,0,0.00,18,"Glob (Var ""b"")",{a|b| } ; { |b| } ; { |b| } ; {a|b|c} ; {a|b|...,choreography


In [40]:
metrics = [
    "trace_len",
    "num_mess",
    "size_mess",
    "nb_progressions"
]

In [41]:
metrics_data = []
for metric in metrics:
    metrics_data.append(select_metric(sdf, metric))

merged_metrics = pd.concat(metrics_data)

(trace_length, num_mess, size_mess, nb_progressions) = merged_metrics

merged_metrics

,formula_id,strategy,value,metric
0,0,orchestration,2.0,trace_len
1,1,orchestration,2.0,trace_len
2,2,orchestration,2.0,trace_len
3,3,orchestration,6.0,trace_len
4,4,orchestration,2.0,trace_len
...,...,...,...,...
76,22,choreography,81.0,nb_progressions
77,23,choreography,191.0,nb_progressions
78,24,choreography,123.0,nb_progressions
79,25,choreography,18.0,nb_progressions


In [42]:
def flatten_once(l: List[List]):
    return [item for sublist in l for item in sublist]

In [43]:
map_ops = lambda x: flatten_once(count_all_ops(x['formula']))

f_ops = sdf.copy()
f_ops['newcolumn'] = f_ops.apply(map_ops, axis=1)
ops = flatten_once(all_operators)
f_ops = pd.DataFrame(f_ops['newcolumn'].to_list(), columns=ops)

f_ops

,True,False,Var,And,Or,Neg,Imp,Iff,Xor,Until,Next,Ev,Glob,Previous,Wuntil
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0,0,2,1,0,0,0,0,0,0,1,1,0,0,0
77,0,0,2,1,0,0,0,0,0,0,1,1,0,0,0
78,0,0,2,1,0,1,0,0,0,0,1,1,0,0,0
79,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [44]:
TRACE_LENGTH = 1001
NODES = 3

In [45]:
map_trace = lambda x: flatten_once(extract_sampled_events(x['trace']))
trace_label = lambda n: [f"t_{x // NODES}_{x % NODES}" for x in range(n)]

t_events = sdf.copy()
t_events['newcolumn'] = t_events.apply(map_trace, axis=1)
ids = trace_label(TRACE_LENGTH * NODES)
t_events = pd.DataFrame(t_events['newcolumn'].to_list(), columns=ids)

t_events

,t_0_0,t_0_1,t_0_2,t_1_0,t_1_1,t_1_2,t_2_0,t_2_1,t_2_2,t_3_0,...,t_997_2,t_998_0,t_998_1,t_998_2,t_999_0,t_999_1,t_999_2,t_1000_0,t_1000_1,t_1000_2
0,a,b,,,,,a,,,,...,c,a,b,c,a,,,a,b,c
1,,,c,a,b,c,a,,c,a,...,c,,,,a,,c,,,c
2,,b,c,a,,,a,,c,,...,c,a,b,c,,,c,a,b,
3,a,b,,a,b,c,,b,,,...,,,,,a,,,a,b,c
4,a,,c,,,,,,,a,...,c,a,b,,,,c,a,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,,,,,,,a,,,a,...,c,,,c,,b,c,a,b,
77,,,c,a,,,,b,c,,...,c,a,,,a,b,,a,,c
78,a,,c,,,,a,,,,...,c,,b,c,,,,a,,
79,a,b,,,b,,,b,,a,...,,a,b,c,,,c,,,c


In [48]:
expanded_df = f_ops.merge(t_events, left_index=True, right_index=True)
expanded_df = sdf.merge(expanded_df, left_index=True, right_index=True)
expanded_df

,formula_id,x,trace_len,num_mess,size_mess,nb_progressions,formula,trace,strategy,True,...,t_997_2,t_998_0,t_998_1,t_998_2,t_999_0,t_999_1,t_999_2,t_1000_0,t_1000_1,t_1000_2
0,0,1,2,3,1.00,3,"Next (Var ""b"")",{a|b| } ; { | | } ; {a| | } ; { | |c} ; { |b|...,orchestration,0,...,c,a,b,c,a,,,a,b,c
1,1,1,2,3,1.00,4,"Ev (Var ""b"")",{ | |c} ; {a|b|c} ; {a| |c} ; {a|b|c} ; { |b|...,orchestration,0,...,c,,,,a,,c,,,c
2,2,1,2,3,1.00,3,"Next (Var ""a"")",{ |b|c} ; {a| | } ; {a| |c} ; { | | } ; {a|b|...,orchestration,0,...,c,a,b,c,,,c,a,b,
3,3,1,6,15,1.00,12,"Glob (Var ""b"")",{a|b| } ; {a|b|c} ; { |b| } ; { |b|c} ; {a|b|...,orchestration,0,...,,,,,a,,,a,b,c
4,4,1,2,3,1.00,4,"Glob (Var ""c"")",{a| |c} ; { | | } ; { | | } ; {a| | } ; {a| |...,orchestration,0,...,c,a,b,,,,c,a,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,22,3,14,13,2.51,81,"Next (Ev (And (Var ""b"", Var ""c"")))",{ | | } ; { | | } ; {a| | } ; {a| |c} ; { | |...,choreography,0,...,c,,,c,,b,c,a,b,
77,23,3,16,15,2.57,191,"Ev (Next (And (Var ""a"", Var ""b"")))",{ | |c} ; {a| | } ; { |b|c} ; { |b|c} ; { | |...,choreography,0,...,c,a,,,a,b,,a,,c
78,24,3,13,11,2.22,123,"Ev (And (Neg (Var ""a""), Next (Var ""c"")))",{a| |c} ; { | | } ; {a| | } ; { | | } ; {a|b|...,choreography,0,...,c,,b,c,,,,a,,
79,25,3,9,0,0.00,18,"Glob (Var ""b"")",{a|b| } ; { |b| } ; { |b| } ; {a|b|c} ; {a|b|...,choreography,0,...,,a,b,c,,,c,,,c


## Plotting

In [ ]:
def plot(data, ax=None):
    """
    Plots the data from a single-strategy dataframe
    :param data: single-strategy dataframe
    :return: nothing. Plots the data
    """
    g = sns.catplot(
        #ax=ax,
        data=data, kind="bar",
        ci="sd", palette="dark", alpha=.8, height=6
    )
    g.set_axis_labels("", "Count")

In [ ]:
# Plotting
plot(sdf)

In [ ]:
g = sns.FacetGrid(merged_metrics, col="metric", aspect=1.5, sharey=False)
g.map(sns.barplot, "strategy", "value", order=["orchestration", "migration", "choreography"])

In [ ]:
ndf = merged_metrics.copy()
ndf['strategy'] = ndf['strategy'].str.replace("^cent_", "1")
ndf['strategy'] = ndf['strategy'].str.replace("^decent_", "2")
ndf['strategy'] = ndf['strategy'].str.replace("^odecent_", "3")
ndf

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(ndf, 'strategy')
